### Implementation of final data science project in IBM Data Science Capstone Course work

In [168]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [169]:
# reading the webpage
from bs4 import BeautifulSoup
import requests
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [170]:
# Getting the class for which we need to display data
class_data = soup.find(class_="wikitable sortable")

In [171]:
# Columns for dataFrame
cols = [x.string.rstrip() for x in class_data.find_all('th')]
cols

['Postcode', 'Borough', 'Neighbourhood']

In [172]:
# Test to read first <tr> tag
tr = class_data.find_all('tr')[3]
print(tr.find_all('td')[0].string)
print(tr.find_all('td')[1].find_all('a')[0].string)
print(tr.find_all('td')[2].find_all('a')[0].string)


M3A
North York
Parkwoods


In [173]:
main_dict = {}
IGNORE = 'Not assigned'
for tr in class_data.find_all('tr')[1:]:
    tds = tr.find_all('td')
    postcode = tds[0].string
    if len(tds[1].find_all('a')) > 0:
        borough = tds[1].find_all('a')[0].string
    else:
        borough = tds[1].string 
        if borough == IGNORE:
            continue
    
    if len(tds[2].find_all('a')) > 0:
        neighbourhood = tds[2].find_all('a')[0].string
    else:
        neighbourhood = tds[2].string.rstrip()
        neighbourhood = borough if neighbourhood == IGNORE else neighbourhood
    
#     print(postcode, borough, neighbourhood)
    postcode, borough, neighbourhood = [x.rstrip() for x in (postcode, borough, neighbourhood)]
    if postcode not in main_dict:
        main_dict[postcode] = {
            'Borough': borough,
            'Neighbourhood': [neighbourhood]
        }
    else:
        main_dict[postcode]['Neighbourhood'].append(neighbourhood)


  

In [174]:
df_dict = {col:[] for col in cols}
for key in main_dict:
    df_dict[cols[0]].append(key)
    df_dict[cols[1]].append(main_dict[key]['Borough'])
    df_dict[cols[2]].append(','.join(main_dict[key]['Neighbourhood'][::-1]))

df = pd.DataFrame(df_dict)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern,Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill,Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District,Ryerson"


In [176]:
to_display = ['M5G', 'M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A']
print(len(to_display))
df_display = df.loc[df.Postcode.map(lambda x: x in to_display)]
df_display.head(12)


12


,Postcode,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
6,M1B,Scarborough,"Malvern,Rouge"
8,M4B,East York,"Parkview Hill,Woodbine Gardens"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Wexford,Maryvale"


In [177]:
df.shape

(103, 3)

In [178]:
filename = 'Geospatial_Coordinates.csv'
location_df = pd.read_csv(filename)
location_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [179]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [180]:
for i in range(df.shape[0]):
    postal_code = df.loc[i,'Postcode']
    latitude = location_df[location_df["Postal Code"] == postal_code]['Latitude'].values
    longitude = location_df[location_df["Postal Code"] == postal_code]['Longitude'].values
    df.loc[i, 'Latitude'] = latitude 
    df.loc[i, 'Longitude'] = longitude


In [181]:
df.head(6)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
